## Librerias

In [1]:
import pandas as pd
import joblib
import os
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import GridSearchCV, KFold , train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor

os.chdir('../')

ModuleNotFoundError: No module named 'pandas'

## Datos entrenamiento

In [2]:
df_train = joblib.load("datos/data_train.joblib")


ModuleNotFoundError: No module named 'pandas.core.indexes.numeric'

## Encoder y selector de variables

In [ ]:
fwiz = joblib.load("modelos/fwiz.joblib")
cat_econder = joblib.load("modelos/cat_econder.joblib")

## Aplicar encoder y selector de variables

In [ ]:
X_train_t = cat_econder.transform(df_train.drop(columns=['clean_valor_total_avaluo']))

In [ ]:
X_train_selected = pd.concat([fwiz.transform(X_train_t), df_train['clean_valor_total_avaluo']], axis=1)

In [ ]:

X = X_train_selected.drop('clean_valor_total_avaluo', axis=1)
y = X_train_selected['clean_valor_total_avaluo']

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Iterar modelos 

In [ ]:
#random forest regressor
# xgboost regressor
#lgbm regressor 
#gradient boosting regressor
#decision tree regressor

In [ ]:
# Definir la cuadrícula de parámetros para cada modelo
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
}

param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

param_grid_lgbm = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

param_grid_dt = {
    'max_depth': [5, 10, 20],
}

In [ ]:
def run_experiment(model, param_grid, model_name, run_name):
    with mlflow.start_run(run_name=run_name):
        # Definir la validación cruzada k-fold
        cv = KFold(n_splits=5, shuffle=True, random_state=42)

        # Utilizar la métrica MAPE en la búsqueda en cuadrícula
        scoring = make_scorer(mean_absolute_error, greater_is_better=False)

        # Realizar la búsqueda en cuadrícula con validación cruzada
        grid_search = GridSearchCV(model, param_grid, scoring=scoring, cv=cv)
        grid_search.fit(X, y)
        
        # Obtener los mejores parámetros y el modelo ajustado
        best_params = grid_search.best_params_
        best_model = grid_search.best_estimator_

        # Calcular la métrica MAPE promedio utilizando validación cruzada
        mape_scores = -grid_search.cv_results_['mean_test_score']
        average_mape = mape_scores.mean()

        # Registrar los resultados en MLflow
        mlflow.log_params(best_params)
        mlflow.log_metric('Average_MAPE', average_mape)
        mlflow.sklearn.log_model(best_model, model_name)


In [ ]:
# Ejecutar experimentos para cada modelo
run_experiment(RandomForestRegressor(), param_grid_rf, 'RandomForestRegressor', run_name='experiment_rf')
run_experiment(XGBRegressor(), param_grid_xgb, 'XGBRegressor', run_name='experiment_xgb')
run_experiment(LGBMRegressor(), param_grid_lgbm, 'LGBMRegressor', run_name='experiment_lgbm')
run_experiment(GradientBoostingRegressor(), param_grid_gb, 'GradientBoostingRegressor', run_name='experiment_gb')
run_experiment(DecisionTreeRegressor(), param_grid_dt, 'DecisionTreeRegressor', run_name='experiment_dt')
